In [ ]:
# Checkpoint 2
# SDS 

# Lucas Paes Landim Pereira - RM550349  
# Gabriela Soares Teixeira - RM98853

In [45]:
import pandas as pd
import scipy.stats as stats
import numpy as np
from scipy.stats import shapiro, levene, f_oneway, friedmanchisquare
import scikit_posthocs as sp

# Carregando os dados
df = pd.read_csv('teste (1).txt', sep='\t', header=None, names=['estado', 'valor'])

# 1. Estimativas de localização e variabilidade para a coluna valor
media = df['valor'].mean()
mediana = df['valor'].median()
variancia = df['valor'].var()
desvio_padrao = df['valor'].std()

print(f'Média: {media}')
print(f'Mediana: {mediana}')
print(f'Variância: {variancia}')
print(f'Desvio Padrão: {desvio_padrao}')


Média: 157.54811583577714
Mediana: 46.485
Variância: 130208.61649316788
Desvio Padrão: 360.84431060107886


In [46]:
# 2. Recalculando as estimativas agrupando-as pela coluna estado
estimativas_estado = df.groupby('estado')['valor'].agg(['mean', 'median', 'var', 'std'])
print(estimativas_estado)

              mean  median            var         std
estado                                               
MG       81.192008  34.240   11264.612340  106.134878
MT      160.087158  45.045   68522.991864  261.768967
PR      170.376980  37.445  332882.766026  576.959935
RJ      167.878026  64.120   75633.052893  275.014641
RS      126.332219  43.450   35064.532132  187.255259
SC      103.789109  34.550   32068.223448  179.076027
SP      178.917040  53.580  104004.155155  322.496752


In [47]:
# 3. Verificando a normalidade dos dados da coluna valor
stat, p = shapiro(df['valor'])
print('Estatística-T =%.3f, p-valor= %.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('A amostra parece Gaussiana (aceita H0)')
else:
    print('A amostra não parece Gaussiana (rejeita H0)')

Estatística-T =0.377, p-valor= 0.000
A amostra não parece Gaussiana (rejeita H0)


c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [48]:
# 4. Avaliando se a variância entre os grupos é estatisticamente diferente
estados = df['estado'].unique()
grupos = [df[df['estado'] == estado]['valor']
          for estado in estados]
stat, p = levene(*grupos)
print('Estatística-T= %.3f, p-valor= %.3f' % (stat, p))
if p > alpha:
    print('As variâncias são iguais (aceita H0)')
else:
    print('As variâncias são diferentes (rejeita H0)')
    
    # Encontrando o tamanho da menor amostra
min_length = min(len(grupo) for grupo in grupos)

# Reduzindo todas as amostras para o menor tamanho da amostra
grupos = [grupo[:min_length] for grupo in grupos]

Estatística-T= 3.837, p-valor= 0.001
As variâncias são diferentes (rejeita H0)


In [49]:
# 5. Teste de hipóteses para determinar se existe diferença significativa entre cada um dos grupos
if p > alpha:
    # ANOVA
    stat, p = f_oneway(*grupos)
else:
    # Qui Quadrado de Friedman
    stat, p = friedmanchisquare(*grupos)
    print('Estatística-t= %.3f, p-valor= %.3f' % (stat, p))
    
if p < 0.01:
    print('Existe diferença entre as amostras com 1% de significância')
elif p < 0.05:
    print('Existe diferença entre as amostras com 5% de significância')
elif p < 0.10:
    print('Existe diferença entre as amostras com 10% de significância')
else:
    print('Não existe diferença entre as amostras')

Estatística-t= 40.918, p-valor= 0.000
Existe diferença entre as amostras com 1% de significância


In [50]:
import scikit_posthocs as sp
# 6. Teste posthoc correspondente da matriz de p-valores do teste de hipóteses realizado no item 5.
# Como o teste posthoc depende do teste de hipóteses usado, este passo pode variar.
# Teste de Tukey HSD após ANOVA:

if p > alpha:
    posthoc_df = sp.posthoc_ttest(grupos)
else:
    posthoc_df = sp.posthoc_wilcoxon(grupos)

print(posthoc_df)


          1         2         3         4         5         6         7
1  1.000000  0.002856  0.008110  0.554425  0.738619  0.993243  0.579662
2  0.002856  1.000000  0.928468  0.000227  0.000773  0.002763  0.000107
3  0.008110  0.928468  1.000000  0.000081  0.000853  0.005166  0.000259
4  0.554425  0.000227  0.000081  1.000000  0.255726  0.558973  0.446941
5  0.738619  0.000773  0.000853  0.255726  1.000000  0.670728  0.581983
6  0.993243  0.002763  0.005166  0.558973  0.670728  1.000000  0.680634
7  0.579662  0.000107  0.000259  0.446941  0.581983  0.680634  1.000000
